In [ ]:
import pieces
import belief
import tasks

import numpy as np
from matplotlib import pyplot as plt

In [ ]:
# skill_to_action = {
#     0: "place_fm",
#     1: "place_buzzer",
#     2: "place_touch_plate",
#     3: "place_reed_switch",
#     4: "",
#     5: PieceSkill("identify_normal_switch", "switch", SkillProbs()),
#     6: PieceSkill("identify_cds", "cds", SkillProbs(prob_guessing=0.1)),
#     7: PieceSkill("identify_led", "led", SkillProbs()),
#     8: PieceSkill("identify_lamp", "lamp", SkillProbs()),
#     9: PieceSkill("identify_battery", "battery", SkillProbs()),
#     10: PieceSkill("identify_speaker", "speaker", SkillProbs()),
#     11: PieceSkill("identify_mc", "mc", SkillProbs(prob_guessing=0.1)),
#     12: PieceSkill("identify_motor", "motor", SkillProbs()),
#     13: Skill("connect_pieces", "connection", SkillProbs(prob_guessing=0.1, prob_learning=0.5)),
#     14: Skill("close_circuit", "closed", SkillProbs(prob_guessing=0.1, prob_slipping=0.3, prob_learning=0.4)),
#     15: Skill("series_circuit", "series", SkillProbs(prob_guessing=0.1, prob_slipping=0.3, prob_learning=0.5)),
#     16: Skill("parallel_circuit", "parallel", SkillProbs(prob_guessing=0.1, prob_slipping=0.3, prob_learning=0.5)),
#     17: StudDirectionSkill("led_pos_stud", "led", StudType.POSITIVE, SkillProbs(prob_guessing=0.5, prob_slipping=0.2, prob_learning=0.6)),
#     18: StudDirectionSkill("led_neg_stud", "led", StudType.NEGATIVE, SkillProbs(prob_guessing=0.5, prob_slipping=0.2, prob_learning=0.6)),
#     19: StudDirectionSkill("fm_in_stud", "fm", StudType.IN, SkillProbs(prob_guessing=0.1, prob_slipping=0.4, prob_learning=0.5)),
#     20: StudDirectionSkill("fm_out_stud", "fm", StudType.OUT, SkillProbs(prob_guessing=0.1, prob_slipping=0.4, prob_learning=0.5)),
#     21: StudDirectionSkill("fm_signal_stud", "fm", StudType.SIGNAL, SkillProbs(prob_guessing=0.1, prob_slipping=0.5, prob_learning=0.5)),
#     22: StudDirectionSkill("mc_in_stud", "mc", StudType.IN, SkillProbs(prob_guessing=0.1, prob_slipping=0.4, prob_learning=0.5)),
#     23: StudDirectionSkill("mc_out_stud", "mc", StudType.OUT, SkillProbs(prob_guessing=0.1, prob_slipping=0.4, prob_learning=0.5)),
#     24: StudDirectionSkill("mc_trigger_stud", "mc", StudType.TRIGGER, SkillProbs(prob_guessing=0.1, prob_slipping=0.5, prob_learning=0.5)),
#     25: StudDirectionSkill("mc_repeat_stud", "mc", StudType.REPEAT, SkillProbs(prob_guessing=0.1, prob_slipping=0.5, prob_learning=0.5)),
#     26: StudDirectionSkill("mc_restart_stud", "mc", StudType.RESTART, SkillProbs(prob_guessing=0.1, prob_slipping=0.5, prob_learning=0.5)),
# }

In [ ]:
def create_beliefs(task, uids, masteries):
    beliefs = {}
    
    for sid in task.sids:
        p_L_0s = {}
        for uid in uids:
            p_L_0s[uid] = masteries[uid][sid]

        skill = tasks.sid_skill_map[sid]
        skill_probs = skill.skill_probs

        skill_belief = belief.GroupSkillBelief(sname=skill.skill_name,
                                        sid=sid, 
                                        uids=task.uids, 
                                        p_L_0s=p_L_0s, 
                                        p_S=skill_probs.prob_slipping, 
                                        p_G=skill_probs.prob_guessing, 
                                        p_T=skill_probs.prob_learning,
                                        n=10, 
                                        discussion_time=task.discussion_time,
                                        solve_time=task.solve_time)
        beliefs[sid] = skill_belief
    return beliefs

In [ ]:
uids = [0, 1]
board = pieces.Board(0)
task = tasks.get_task("task1", uids)

masteries = {}

for uid in uids:
    masteries[uid] = {}
    for sid in tasks.sids:
        masteries[uid][sid] = np.random.randint(2)

beliefs = create_beliefs(task, uids, masteries)


In [ ]:
time = 0

working_on = {}
for sid in task.sids:
    working_on[sid] = {}
    for uid in uids:
        working_on[uid] = 0

while True:

    for sid in task.sids:
        skill_probs = tasks.sid_skill_map[sid].skill_probs

        for uid in uids:
            works_on_prob = masteries[uid][sid] * (1 - skill_probs.probs_slipping) + 1 - (masteries[uid][sid]) * skill_probs.prob_guessing
            works_on = np.random.choice([0, 1], p=[1 - works_on_prob, works_on_prob])
            working_on[sid][uid] += works_on

    if time > task.discussion_time + task.solve_time:
        break
    time += 1